# Setup

In [1]:
from typing import Dict, TypedDict, Optional
from langgraph.graph import StateGraph, END
from langchain.llms import OpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser
import random
import time

import os

In [2]:
llm = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

C:\Users\chanho\AppData\Local\Temp\ipykernel_16220\1936120068.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


# Simulation - 2 Agents

The code was entirely reused from the example by [Mehul Gupta](https://medium.com/data-science-in-your-pocket/multi-agent-conversation-debates-using-langgraph-and-langchain-9f4bf711d8ab) in his Medium's blogpost and the guide provided by [Vadym Barda](https://github.com/langchain-ai/langgraph/tree/main)'s LangGraph Github repository , with a few minor tweaks and proper configuration of the prompts.


In [3]:
# Debate Topic
debate_topic= "Should the United Kingdom continue providing weapons to Israel?"

In [ ]:
# Generate the classes
output_parser = CommaSeparatedListOutputParser()
output = llm("I wish to have a debate on {}. What would be the fighting sides called? Output just the names and nothing else as comma separated list".format(debate_topic))
classes = output_parser.parse(output)

In [4]:
# Generate the classes
classes = ['UK Labour Party', 'UK Conservative Party']
classes[1]

'UK Conservative Party'

## Create Graph

In [12]:
# Defining state variables in Graph and workflow object
class GraphState(TypedDict):
    classification: Optional[str] = None
    history: Optional[str] = None
    current_response: Optional[str] = None
    count: Optional[int]=None
    results: Optional[str]=None
    greeting: Optional[str]=None

workflow = StateGraph(GraphState)

## Simulation Nodes

In [13]:
# Create Graph Nodes

prefix_start= 'You are in support of {} . You are in a debate with {} over the topic: {}. This is the conversation so far \n{}\n. Put forth your next argument to support {} countering {}.\ Dont repeat your previous arguments. Give a short, one line answer.'

def classify(question):
    return llm("classify the sentiment of input as {} or {}. Output just the class. Input:{}".format('_'.join(classes[0].split(' ')),'_'.join(classes[1].split(' ')),question)).strip()

def classify_input_node(state):
    question = state.get('current_response')
    classification = classify(question)  # Assume a function that classifies the input
    return {"classification": classification}

def handle_greeting_node(state):
    return {"greeting": "Hello! Today we will witness the fight between {} vs {}".format(classes[0],classes[1])}

def handle_pro(state):
    summary = state.get('history', '').strip()
    current_response = state.get('current_response', '').strip()
    if summary=='Nothing':
        prompt = prefix_start.format(classes[0],classes[1],debate_topic,'Nothing',classes[0],"Nothing")
        argument = classes[0] +":"+ llm(prompt)
        summary = 'START\n'
    else:
        prompt = prefix_start.format(classes[0],classes[1],debate_topic,summary,classes[0],current_response)
        argument = classes[0] +":"+ llm(prompt)
    return {"history":summary+'\n'+argument,"current_response":argument,"count":state.get('count')+1}

def handle_opp(state):
    summary = state.get('history', '').strip()
    current_response = state.get('current_response', '').strip()
    prompt = prefix_start.format(classes[1],classes[0],debate_topic,summary,classes[1],current_response)
    argument = classes[1] +":"+ llm(prompt)
    return {"history":summary+'\n'+argument,"current_response":argument,"count":state.get('count')+1}    

def result(state):
    summary = state.get('history').strip()
    prompt = "Summarize the conversation and judge who won the debate.No ties are allowed. Conversation:{}".format(summary)
    return {"results":llm(prompt)}

workflow.add_node("classify_input", classify_input_node)
workflow.add_node("handle_greeting", handle_greeting_node)
workflow.add_node("handle_pro", handle_pro)
workflow.add_node("handle_opp", handle_opp)
workflow.add_node("result", result)


<>:3: SyntaxWarning: invalid escape sequence '\ '
<>:3: SyntaxWarning: invalid escape sequence '\ '
C:\Users\chanho\AppData\Local\Temp\ipykernel_16220\2450468759.py:3: SyntaxWarning: invalid escape sequence '\ '
  prefix_start= 'You are in support of {} . You are in a debate with {} over the topic: {}. This is the conversation so far \n{}\n. Put forth your next argument to support {} countering {}.\ Dont repeat your previous arguments. Give a short, one line answer.'


## Simulation Workflow & Connections

Define the round/length of debates

In [14]:
# Define Conditional Edges
def decide_next_node(state):
    return "handle_opp" if state.get('classification')=='_'.join(classes[0].split(' ')) else "handle_pro"

# Define Length of Debate
def check_conv_length(state):
    return "result" if state.get("count")==10 else "classify_input"           # ---------------------> Define X rounds of debate

# Adding Conditional Edges
workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "handle_pro": "handle_pro",
        "handle_opp": "handle_opp"
    }
)

workflow.add_conditional_edges(
    "handle_pro",
    check_conv_length,
    {
        "result": "result",
        "classify_input": "classify_input"
    }
)

workflow.add_conditional_edges(
    "handle_opp",
    check_conv_length,
    {
        "result": "result",
        "classify_input": "classify_input"
    }
)

# Adding graph entry point and remaining definite edges
workflow.set_entry_point("handle_greeting")
workflow.add_edge('handle_greeting', "handle_pro")
workflow.add_edge('result', END)

## Run Simulation

In [15]:
app = workflow.compile()
conversation = app.invoke({'count':0,'history':'Nothing','current_response':''})

C:\Users\chanho\AppData\Local\Temp\ipykernel_16220\2450468759.py:21: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  argument = classes[0] +":"+ llm(prompt)


## Results

In [16]:
print(conversation['history'])

START

UK Labour Party:

"Continuing to provide weapons to Israel only perpetuates the cycle of violence and oppression against Palestinians."
UK Labour Party:

"Supporting Israel's military only fuels their human rights abuses against Palestinians."
UK Conservative Party:

"Providing weapons to Israel helps defend against terrorist threats in the region."
UK Labour Party:
"Arming Israel only escalates tensions and further destabilizes the region."
UK Conservative Party:

"Israel has a right to defend itself against hostile forces in the region."
UK Labour Party: 

"Providing weapons to Israel does not address the root causes of conflict and only perpetuates the suffering of innocent civilians."
UK Labour Party:

"Arming Israel goes against our party's values of promoting peace and human rights."
UK Conservative Party:

"Providing weapons to Israel also supports their right to self-determination and sovereignty."
UK Labour Party: 

"Supporting Israel's military actions only further und

In [17]:
print(conversation["results"])



The conversation revolves around the topic of whether or not the UK should continue to provide weapons to Israel. The UK Labour Party argues that doing so only perpetuates the cycle of violence and human rights abuses against Palestinians, and goes against their party's values of promoting peace and human rights. They also believe that supporting Israel's military only escalates tensions and destabilizes the region. On the other hand, the UK Conservative Party argues that providing weapons to Israel helps defend against terrorist threats and supports their right to self-determination and sovereignty. They also believe that arming Israel helps maintain a balance of power in the region and prevent larger conflicts. 

Overall, it is difficult to determine who "won" the debate as both parties presented valid arguments from their perspectives. However, in terms of promoting peace and human rights, the UK Labour Party may have had a stronger argument. Ultimately, it is up to individuals to

# Simulation - 3 Agents

In [18]:
# Define the debate topic
debate_topic= "Should the United Kingdom continue providing weapons to Israel?"

In [19]:
# Generate the classes
classes = ['UK Labour Party', 'UK Conservative Party', 'UK Liberal Democrats Party']
classes[1]

'UK Conservative Party'

## Create Graph

In [20]:
# Defining state variables in Graph and workflow object
class GraphState(TypedDict):
    classification: Optional[str] = None
    
    history: Optional[str] = None
    current_response: Optional[str] = None
    count: Optional[int]=None
    results: Optional[str]=None
    greeting: Optional[str]=None

workflow = StateGraph(GraphState)

## Simulation Nodes

In [21]:
# Simulation Node

prefix_start = 'You are in support of {}. You are in a debate with {} and {} over the topic: {}. This is the conversation so far \n{}\n. Put forth your next argument to support {} countering {} and {}. Don\'t repeat your previous arguments. Give a short, one line answer.'

# Classify the input
def classify(question):
    return llm("classify the sentiment of input as {}, {}, or {}. Output just the class. Input:{}".format('_'.join(classes[0].split(' ')), 
                                                                                                          '_'.join(classes[1].split(' ')), 
                                                                                                          '_'.join(classes[2].split(' ')), question)).strip()

def classify_input_node(state):
    question = state.get('current_response')
    classification = classify(question)  # Assume a function that classifies the input
    return {"classification": classification}

# Generates a greeting message
def handle_greeting_node(state):
    return {"greeting": "Hello! Today we will witness the debate between {}, {}, and {}".format(classes[0], 
                                                                                                classes[1], 
                                                                                                classes[2])}
### Manages the state for the "Labour Party" side
def handle_Lab(state):
    summary = state.get('history', '').strip()
    current_response = state.get('current_response', '').strip()
    if summary == 'Nothing':
        prompt = prefix_start.format(classes[0], classes[1], classes[2], debate_topic, 'Nothing', classes[0], "Nothing", "Nothing")
        argument = classes[0] + ":" + llm(prompt)
        summary = 'START\n'
    else:
        prompt = prefix_start.format(classes[0], classes[1], classes[2], debate_topic, summary, classes[0], current_response, current_response)
        argument = classes[0] + ":" + llm(prompt)
    return {"history": summary + '\n' + argument, "current_response": argument, "count": state.get('count') + 1}

### Manages the state for the "Conservative Party" side
def handle_Cons(state):
    summary = state.get('history', '').strip()
    current_response = state.get('current_response', '').strip()
    prompt = prefix_start.format(classes[1], classes[0], classes[2], debate_topic, summary, classes[1], current_response, current_response)
    argument = classes[1] + ":" + llm(prompt)
    return {"history": summary + '\n' + argument, "current_response": argument, "count": state.get('count') + 1}

### Manages the state for the "Liberal Democrats Party" side
def handle_LibDem(state):
    summary = state.get('history', '').strip()
    current_response = state.get('current_response', '').strip()
    prompt = prefix_start.format(classes[2], classes[0], classes[1], debate_topic, summary, classes[2], current_response, current_response)
    argument = classes[2] + ":" + llm(prompt)
    return {"history": summary + '\n' + argument, "current_response": argument, "count": state.get('count') + 1}

# Summarizes the Conversation + Judges the Winner
def result(state):
    summary = state.get('history').strip()
    prompt = "Summarize the conversation and judge who won the debate. No ties are allowed. Conversation:{}".format(summary)
    return {"results": llm(prompt)}

# Adding nodes to the Workflow
workflow.add_node("classify_input", classify_input_node)
workflow.add_node("handle_greeting", handle_greeting_node)
workflow.add_node("handle_Lab", handle_Lab)
workflow.add_node("handle_Cons", handle_Cons)
workflow.add_node("handle_LibDem", handle_LibDem)
workflow.add_node("result", result)

## Simulation Workflow & Connections

In [22]:
# Determines the next node based on the classification
def decide_next_node(state):
    if state.get('classification') == '_'.join(classes[0].split(' ')):
        return "handle_Lab"
    elif state.get('classification') == '_'.join(classes[1].split(' ')):
        return "handle_Cons"
    else:
        return "handle_LibDem"

# Checks if the conversation has reached the maximum length
def check_conv_length(state):
    return "result" if state.get("count")== 9 else "classify_input"     # Set Debate Length 

# Add Conditional Transitions 
workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "handle_Lab": "handle_Lab",
        "handle_Cons": "handle_Cons",
        "handle_LibDem": "handle_LibDem"
    }
)

workflow.add_conditional_edges(
    "handle_Lab",
    check_conv_length,                   # determine whether to transition to "result" or back to "classify_input"
    {
        "result": "result",
        "classify_input": "classify_input"
    }
)

workflow.add_conditional_edges(
    "handle_Cons",
    check_conv_length,                   # determine whether to transition to "result" or back to "classify_input"

    {
        "result": "result",
        "classify_input": "classify_input"
    }
)

workflow.add_conditional_edges(
    "handle_LibDem",
    check_conv_length,                   # determine whether to transition to "result" or back to "classify_input"
    {
        "result": "result",
        "classify_input": "classify_input"
    }
)

# Setting the Entry point + Ending point of the workflow
workflow.set_entry_point("handle_greeting")
workflow.add_edge('handle_greeting', "handle_Lab")
workflow.add_edge('result', END)

## Run Simulation

In [23]:
app = workflow.compile()
conversation = app.invoke({'count':0,'history':'Nothing','current_response':''})

## Results

In [24]:
print(conversation['history'])

START

UK Labour Party:

The UK Labour Party believes in promoting peace and stability in the Middle East, and providing weapons to Israel only fuels the conflict.
UK Liberal Democrats Party:

The UK Liberal Democrats Party believes in a balanced approach to the conflict and supports diplomatic solutions.
UK Liberal Democrats Party:

Providing weapons only escalates the violence and hinders the possibility of peace negotiations.
UK Liberal Democrats Party:

The UK Liberal Democrats Party believes in promoting non-violent solutions to conflicts.
UK Liberal Democrats Party:

Providing weapons to Israel only perpetuates the cycle of violence and undermines efforts for a peaceful resolution.
UK Liberal Democrats Party: 


Arming Israel may also have unintended consequences and could potentially harm innocent civilians in the region.
UK Liberal Democrats Party:

Providing weapons to Israel goes against our values of promoting human rights and protecting innocent lives.
UK Liberal Democrats 

In [25]:
print(conversation['results'])



The overall conversation centered around the issue of providing weapons to Israel and how it impacts the conflict in the Middle East. The UK Labour Party argued against arming Israel, stating that it only fuels the conflict and goes against their values of promoting peace and stability. On the other hand, the UK Liberal Democrats Party argued for a more balanced approach, emphasizing the importance of non-violent solutions and the potential harm that could come from providing weapons. They also suggested focusing on diplomatic solutions and promoting peace talks between the two sides. 

Based on the arguments presented, it can be concluded that the UK Liberal Democrats Party won the debate. They were able to provide a more comprehensive and nuanced perspective, highlighting the potential consequences of arming Israel and offering alternative solutions that align with their values. The UK Labour Party's argument, while valid, was not as strong and did not address the potential drawbac

# Simulation - Multiple Agents

The code was entirely reused from the example by [Chester](https://medium.com/data-science-in-your-pocket/multi-agent-conversation-debates-using-langgraph-and-langchain-9f4bf711d8ab)'s notebook in Langchain Github respository, with a few minor tweaks and proper configuration of the prompts.

In [ ]:
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)